In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
from torch.utils.data import random_split, DataLoader, Dataset
import os
from PIL import Image

In [4]:
class Food101Dataset(Dataset):
  def __init__(self, txt_file, root_dir, classes_file, transform=None, max_items=None):
          with open(txt_file, 'r') as f:
              lines = f.read().splitlines()
              self.img_labels = lines[:max_items] if max_items else lines

          with open(classes_file, 'r') as f:
              self.classes = [line.strip() for line in f]
          self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
          self.root_dir = root_dir
          self.transform = transform


  def __len__(self):
      return len(self.img_labels)

  def __getitem__(self, idx):
      rel_path = self.img_labels[idx]
      img_path = os.path.join(self.root_dir, rel_path + ".jpg")
      label_str = rel_path.split("/")[0]
      label = self.class_to_idx[label_str]

      try:
          image = Image.open(img_path).convert("RGB")
      except FileNotFoundError:
          raise RuntimeError(f"Nie znaleziono pliku: {img_path}")

      if self.transform:
          image = self.transform(image)
      return image, label

In [6]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = Food101Dataset(
    txt_file="/content/drive/MyDrive/food-101/meta/train.txt",
    root_dir="/content/drive/MyDrive/food-101/images",
    classes_file="/content/drive/MyDrive/food-101/meta/classes.txt",
    transform=transform,
    max_items=5000  # tylko pierwsze 5000 przykładów
)
train_subset, val_subset = random_split(dataset, [4500, 500])
trainloader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)
validloader = DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=2)

test_dataset = Food101Dataset(
    txt_file="/content/drive/MyDrive/food-101/meta/test.txt",
    root_dir="/content/drive/MyDrive/food-101/images",
    classes_file="/content/drive/MyDrive/food-101/meta/classes.txt",
    transform=transform,
    max_items=5000  # np. ogranicz test na początek
)

testloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models.feature_extraction import create_feature_extractor

class MiniNet(nn.Module):
    def __init__(self, num_classes=101):  # Food-101
        super().__init__()
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.feature_extractor = create_feature_extractor(
            resnet,
            return_nodes={'avgpool': 'features'}
            )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),  # Zapobiega overfitting
            nn.Linear(512, num_classes))

    def forward(self, x):
        features = self.feature_extractor(x)['features']
        return self.classifier(features)


In [14]:
!pip install lightning
import lightning as L
import torchmetrics
import torch.optim as optim

class LitClassifier(L.LightningModule):
    def __init__(self, model, num_classes):
        super().__init__()
        self.model = model
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fn(y_hat, y)
        acc = self.accuracy(y_hat.argmax(1), y)
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def test_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self.model(x)
      loss = self.loss_fn(y_hat, y)
      acc = self.accuracy(y_hat.argmax(1), y)
      self.log("test_loss", loss)
      self.log("test_acc", acc)


    def configure_optimizers(self):
      optimizer = optim.Adam(self.parameters(), lr=1e-3)
      scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
      return [optimizer], [scheduler]


In [ ]:
model = MiniNet(num_classes=101)
classifier = LitClassifier(model=model, num_classes=101)

trainer = L.Trainer(max_epochs=5, accelerator="auto", devices=1)
trainer.fit(classifier, trainloader, validloader)


INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | model    | MiniNet            | 11.5 M | train
1 | loss_fn  | CrossEntropyLoss   | 0      | train
2 | accuracy | MulticlassAccuracy | 0      | trai

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.test(model=classifier, dataloaders=testloader)

In [1]:
model_dir = os.path.join(os.getcwd(), "model")
os.makedirs(model_dir, exist_ok=True)
torch.save(model, os.path.join(model_dir, "food_matching_model.pt"))  # Zapisuje CAŁY obiekt

NameError: name 'os' is not defined

In [2]:
img, label = test_dataset[0]
model.eval()
with torch.no_grad():
    output = model(img.unsqueeze(0))
    pred = output.argmax(1).item()

print(f"Prawdziwa etykieta: {label}, przewidziana: {pred}")
class_names = test_dataset.classes
print("Prawidłowa:", class_names[label])
print("Przewidziana:", class_names[pred])



NameError: name 'test_dataset' is not defined

In [3]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

# Upewnij się, że model jest w trybie ewaluacji
model.eval()

# 10 obrazków do testu
num_images = 10

plt.figure(figsize=(15, 8))
for i in range(num_images):
    img, label = test_dataset[i]
    input_img = img.unsqueeze(0)  # dodaj batch dimension

    with torch.no_grad():
        output = model(input_img)
        pred = output.argmax(1).item()

    # Denormalizacja do wyświetlania
    img_to_show = img * 0.5 + 0.5  # odwrotność normalizacji

    plt.subplot(2, 5, i+1)
    plt.imshow(img_to_show.permute(1, 2, 0).numpy())
    title = f"GT: {class_names[label]}\nPred: {class_names[pred]}"
    color = 'green' if label == pred else 'red'
    plt.title(title, color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()



NameError: name 'model' is not defined